In [7]:
import os
import azureml.core
import pandas as pd
from azureml.core.runconfig import JarLibrary
from azureml.core.compute import ComputeTarget, DatabricksCompute
from azureml.exceptions import ComputeTargetException
from azureml.core.datastore import Datastore
from azureml.data.data_reference import DataReference
from azureml.core.databricks import PyPiLibrary

from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.core import Workspace, Environment, Experiment, Datastore, Dataset, ScriptRunConfig
from azureml.pipeline.core import Pipeline, PipelineData, TrainingOutput
from azureml.pipeline.steps import DatabricksStep, PythonScriptStep
from azureml.train.hyperdrive import choice, loguniform

from sklearn.model_selection import train_test_split
from azureml.train.automl import AutoMLConfig

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)


SDK version: 1.44.0


In [8]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')


nlp-workspace
openaml
eastus2
f9b97038-ed78-4a26-a1a7-51e81e75d867


In [9]:

from azureml.core import Workspace, Experiment, Environment, Model, Dataset, Run

run = ws.get_run(run_id="7f2fc837-287f-45ce-8b42-40c64771cdff")
exp = run.experiment

print('Run {} will be used'.format(run))
print('Experiment {} will be used'.format(exp))


Run Run(Experiment: transformer_hp,
Id: 7f2fc837-287f-45ce-8b42-40c64771cdff,
Type: azureml.StepRun,
Status: Failed) will be used
Experiment Experiment(Name: transformer_hp,
Workspace: nlp-workspace) will be used


In [10]:
all_runs = exp.get_runs(include_children=True)

print(f'len(all_runs) = {len(list(all_runs))}')

TypeError: object of type 'generator' has no len()

In [ ]:
source_directory = "./project"

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "gpucluster"
compute_target = ComputeTarget(workspace=ws, name=cluster_name)

In [ ]:
from datetime import datetime

timenow = datetime.now().strftime('%Y-%m-%d-%H-%M')

pipeline_name = f"Sentiment-Classifier-{timenow}-Pipeline"
print(pipeline_name)

published_pipeline = pipeline.publish(
    name=pipeline_name, 
    description=pipeline_name)
print("Newly published pipeline id: {}".format(published_pipeline.id))